<a href="https://colab.research.google.com/github/hamednasr/DDoS-Attacks/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import sklearn.ensemble   
import sklearn.datasets   
from sklearn import metrics
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

## Reading the Data

In [ ]:
data = pd.read_csv('KDDTrain.csv')
data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


## Filtering the Data

In [ ]:

rows, columns = np.where(pd.isnull(data))
data_filtered = data.drop(labels=rows)

print("There are %d rows having null values" %(len(data) - len(data_filtered)))

data_filtered

There are 0 rows having null values


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


# Removing Duplicated Data

In [ ]:
data_unduplicated = data_filtered.drop_duplicates()
print("There are %d duplicated rows" %(len(data_filtered) - len(data_unduplicated)))
data_unduplicated

There are 6356 duplicated rows


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:
X, y = data_unduplicated.iloc[:, :-1], data_unduplicated.iloc[:, -1]
print("Shape of X: %s" %str(X.shape), "- Type of X: %s" %str(type(X)))
print("Shape of y: %s" %str(y.shape), "- Type of y: %s" %str(type(y)))
y

Shape of X: (119616, 38) - Type of X: <class 'pandas.core.frame.DataFrame'>
Shape of y: (119616,) - Type of y: <class 'pandas.core.series.Series'>


0          normal
1         neptune
2          normal
3          normal
4         neptune
           ...   
125967    neptune
125968     normal
125969     normal
125970    neptune
125971     normal
Name: Label, Length: 119616, dtype: object

# Labeling Data

In [ ]:

print("The number of original classes: %d" %len(y.unique()))
y.unique()

The number of original classes: 23


array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [ ]:

y_array = np.array(y)
y_encoded = np.zeros((y.shape))
for i in range(len(y_array)):
    if y_array[i] in ['normal']:
        y_array[i] = 'normal'
        y_encoded[i] = 0
    elif y_array[i] in ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']:
        y_array[i] = 'DOS'
        y_encoded[i] = 1
    elif y_array[i] in ['ipsweep', 'nmap', 'portsweep', 'satan']:
        y_array[i] = 'probing'
        y_encoded[i] = 2
    elif y_array[i] in ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']:
        y_array[i] = 'R2L'
        y_encoded[i] = 3
    elif y_array[i] in ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']:
        y_array[i] = 'U2R'
        y_encoded[i] = 4
y_df = pd.Series(y_array)
y_df.unique()

array(['normal', 'DOS', 'R2L', 'probing', 'U2R'], dtype=object)

In [ ]:
for name in y_df.unique():
    print("The number of samples of class %s: %d" %(str(name), len(X[y_array==name])))
y_encoded[:10]

The number of samples of class normal: 67292
The number of samples of class DOS: 39893
The number of samples of class R2L: 995
The number of samples of class probing: 11384
The number of samples of class U2R: 52


array([0., 1., 0., 0., 1., 1., 1., 1., 1., 1.])

# Sorting Data

In [ ]:

data_new = X
data_new['Label'] = y_encoded
data_sorted = data_new.sort_values(by='Label')
data_sorted

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.0,0.0,0.00,0.0,0.0
71814,0,241,4198,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.00,0.00,1.00,2,255,1.00,0.00,0.50,0.04,0.0,0.0,0.00,0.0,0.0
71813,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.00,12,93,1.00,0.00,0.08,0.34,0.0,0.0,1.00,1.0,0.0
71812,0,45,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,260,0.0,0.0,0.0,0.0,1.00,0.00,0.01,103,200,0.85,0.02,0.85,0.01,0.0,0.0,0.11,0.0,0.0
71810,0,206,12090,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,9,0.0,0.0,0.0,0.0,1.00,0.00,0.33,6,255,1.00,0.00,0.17,0.04,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84225,0,0,5928,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,2,82,1.00,0.00,1.00,0.02,0.0,0.0,0.00,0.0,4.0
99759,158,1567,3095,0,0,0,3,0,1,4,1,0,0,1,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.0,0.0,0.00,0.0,4.0
108499,85,277,693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.0,0.0,0.00,0.0,4.0
82251,7,226,698,0,0,0,4,0,1,0,0,0,0,4,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.0,0.0,0.00,0.0,4.0


In [ ]:
data_array = data_sorted.to_numpy()
X_new, y_new = data_array[:, :-1], data_array[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=0)

In [ ]:
feature_selection = SelectKBest(chi2, k=10).fit(X_train, y_train)
X_train_new = feature_selection.transform(X_train)
X_test_new = feature_selection.transform(X_test)
print(feature_selection.get_support())

[ True  True  True False False False  True False False False False False
 False False False False False False False  True  True False  True False
 False False False False  True  True False False False False False  True
 False False]


# Grid Search

In [ ]:
param_rand_forest = {'n_estimators':[np.random.randint(1, 1000)]}
mfcc_forest = sklearn.ensemble.RandomForestClassifier()

rand_mfcc_forest = GridSearchCV(mfcc_forest, param_rand_forest).fit(X_train, y_train)
rand_mfcc_forest

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

# Training, Accuracy and Reports

In [ ]:
rand = RandomForestClassifier(n_estimators=100).fit(X_train_new, y_train)


print("Test Accuracy:",metrics.accuracy_score(y_test, rand.predict(X_test_new))*100, chr(37))
print("Train Accuracy:",metrics.accuracy_score(y_train, rand.predict(X_train_new))*100, chr(37))

Test Accuracy: 99.43989299448252 %
Train Accuracy: 99.79726622915186 %


In [ ]:
y_tst_pred = rand.predict(X_test_new)
print(confusion_matrix(y_test, y_tst_pred))
print("==================================")
print(classification_report(y_test, y_tst_pred))

[[13416     4    19     2     0]
 [    9  7952    39     0     0]
 [   15    33  2213     0     0]
 [    8     0     0   205     1]
 [    3     0     1     0     4]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     13441
         1.0       1.00      0.99      0.99      8000
         2.0       0.97      0.98      0.98      2261
         3.0       0.99      0.96      0.97       214
         4.0       0.80      0.50      0.62         8

    accuracy                           0.99     23924
   macro avg       0.95      0.89      0.91     23924
weighted avg       0.99      0.99      0.99     23924



In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train_scaled = std_scaler.fit_transform(X_train_new)
X_test_scaled = std_scaler.fit_transform(X_test_new)
X_train_scaled.shape

(95692, 10)

In [ ]:
param_rand_forest = {'n_estimators':[np.random.randint(1, 1000)]}
mfcc_forest = sklearn.ensemble.RandomForestClassifier()

rand_mfcc_forest = GridSearchCV(mfcc_forest, param_rand_forest).fit(X_train_scaled, y_train)
rand_mfcc_forest.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=879,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rand = RandomForestClassifier(n_estimators=879).fit(X_train_scaled, y_train)


print("Test Accuracy:",metrics.accuracy_score(y_test, rand.predict(X_test_scaled))*100, chr(37))
print("Train Accuracy:",metrics.accuracy_score(y_train, rand.predict(X_train_scaled))*100, chr(37))

Test Accuracy: 76.52148470155493 %
Train Accuracy: 99.79726622915186 %


In [ ]:
y_tst_pred = rand.predict(X_test_scaled)
print(confusion_matrix(y_test, y_tst_pred))
print("==================================")
print(classification_report(y_test, y_tst_pred))